In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, transforms
from tqdm import tqdm

In [2]:
import numpy as np
import torchvision
from torchvision import datasets, transforms

X_train = datasets.FashionMNIST(root='./data', train=True, download=True,
                         transform=transforms.Compose([transforms.ToTensor()]))

idx = np.append(np.where(X_train.targets == 0)[0], 
                np.where(X_train.targets == 1)[0])
idx = np.append(idx, 
                np.where(X_train.targets == 2)[0])
idx = np.append(idx, 
                np.where(X_train.targets == 3)[0])
idx = np.append(idx, 
                np.where(X_train.targets == 4)[0])
idx = np.append(idx, 
                np.where(X_train.targets == 5)[0])
idx = np.append(idx, 
                np.where(X_train.targets == 6)[0])
idx = np.append(idx, 
                np.where(X_train.targets == 7)[0])


X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]


train_loader = torch.utils.data.DataLoader(X_train, batch_size=128, shuffle=True, pin_memory=True)

In [3]:
X_test = datasets.FashionMNIST(root='./data', train=False, download=True,
                        transform=transforms.Compose([transforms.ToTensor()]))

idx = np.append(np.where(X_test.targets == 0)[0], 
                np.where(X_test.targets == 1)[0])
idx = np.append(idx, 
                np.where(X_test.targets == 2)[0])
idx = np.append(idx, 
                np.where(X_test.targets == 3)[0])
idx = np.append(idx, 
                np.where(X_test.targets == 4)[0])
idx = np.append(idx, 
                np.where(X_test.targets == 5)[0])
idx = np.append(idx, 
                np.where(X_test.targets == 6)[0])
idx = np.append(idx, 
                np.where(X_test.targets == 7)[0])


X_test.data = X_test.data[idx]
X_test.targets = X_test.targets[idx]

test_loader = torch.utils.data.DataLoader(X_test, batch_size=128, shuffle=True)

In [4]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=8):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(384 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 384 * 7 * 7)
        x = self.classifier(x)
        return x


In [5]:
class LeNet5(nn.Module):
    def __init__(self, num_classes=8):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features=16*4*4, out_features=120),
            nn.Tanh(),
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=num_classes)
        )
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(-1, 16*4*4)  # Flatten the tensor
        logits = self.classifier(x)
        return logits


In [6]:
# Define the ensemble model
class Ensemble(nn.Module):
    def __init__(self, modelA, modelB):
        super(Ensemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB

    def forward(self, x):
        outA = self.modelA(x)
        outB = self.modelB(x)
        # Average the outputs
        return (outA + outB) / 2

In [7]:
device = torch.device("cpu")

# Instantiate models
alexnet_model = AlexNet(num_classes=8).to(device)  
lenet_model = LeNet5(num_classes=8).to(device)     

In [8]:
# Create ensemble
model = Ensemble(alexnet_model, lenet_model).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [9]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of trainable parameters:", total_params)

Number of trainable parameters: 94708136


In [10]:
best_accuracy = 0
best_model = None

train_losses = []
test_losses = []

for epoch in range(10):
    model.train()
    train_loss = 0
    for idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
    
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for idx, (data, target) in enumerate(tqdm(test_loader)):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    accuracy = 100. * correct / len(test_loader.dataset)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model.state_dict()

    print(f"Epoch {epoch + 1}/{10}")
    print(f"Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")


100%|███████████████████████████████████████████| 63/63 [00:07<00:00,  7.88it/s]


Epoch 1/10
Train Loss: 0.0042, Test Loss: 0.0028, Accuracy: 86.50%


100%|███████████████████████████████████████████| 63/63 [00:07<00:00,  8.39it/s]


Epoch 2/10
Train Loss: 0.0026, Test Loss: 0.0025, Accuracy: 88.75%


100%|███████████████████████████████████████████| 63/63 [00:08<00:00,  7.28it/s]


Epoch 3/10
Train Loss: 0.0022, Test Loss: 0.0023, Accuracy: 89.50%


100%|███████████████████████████████████████████| 63/63 [00:07<00:00,  8.36it/s]


Epoch 4/10
Train Loss: 0.0020, Test Loss: 0.0022, Accuracy: 89.85%


100%|███████████████████████████████████████████| 63/63 [00:07<00:00,  8.40it/s]


Epoch 5/10
Train Loss: 0.0018, Test Loss: 0.0020, Accuracy: 91.00%


100%|███████████████████████████████████████████| 63/63 [00:07<00:00,  8.32it/s]


Epoch 6/10
Train Loss: 0.0016, Test Loss: 0.0020, Accuracy: 90.91%


100%|███████████████████████████████████████████| 63/63 [00:07<00:00,  8.38it/s]


Epoch 7/10
Train Loss: 0.0015, Test Loss: 0.0020, Accuracy: 91.06%


100%|███████████████████████████████████████████| 63/63 [00:07<00:00,  8.36it/s]


Epoch 8/10
Train Loss: 0.0014, Test Loss: 0.0020, Accuracy: 91.24%


100%|███████████████████████████████████████████| 63/63 [00:08<00:00,  7.76it/s]


Epoch 9/10
Train Loss: 0.0013, Test Loss: 0.0020, Accuracy: 91.61%


100%|███████████████████████████████████████████| 63/63 [00:07<00:00,  8.46it/s]

Epoch 10/10
Train Loss: 0.0012, Test Loss: 0.0020, Accuracy: 91.79%


In [11]:
print(f"Best Accuracy: {best_accuracy:.2f}%")
torch.save(best_model, "ensemble fashionmnist 0-7.pth")

Best Accuracy: 91.79%


/tmp/ipykernel_12035/153970709.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


In [12]:
import matplotlib.pyplot as plt

# Use a professional style
plt.style.use('seaborn-darkgrid')

# Create a color palette
color_palette = plt.get_cmap('Set1')

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting both the curves simultaneously with enhanced aesthetics
ax.plot(train_losses, color=color_palette(0), marker='o', linestyle='-', linewidth=2, label='Training loss')
ax.plot(test_losses, color=color_palette(1), marker='', linestyle='--', linewidth=2, label='Validation loss')

# Naming the x-axis, y-axis and the whole graph with enhanced font properties
ax.set_xlabel("Epochs", fontsize=14, fontweight='bold')
ax.set_ylabel("Loss", fontsize=14, fontweight='bold')
ax.set_title("Training and Validation Loss Curves", fontsize=16, fontweight='bold')

# Displaying the legend with larger font size
ax.legend(fontsize=12)

# Enhance tick visibility and size
ax.tick_params(axis='both', which='major', labelsize=12)

# Adjust the plot for better layout
plt.tight_layout()

# Saving the graph in high quality
plt.savefig("ensemble fashionmnist 0-7 loss curve.png", dpi=300)

# Displaying the graph
plt.show()
